## Getting started with LoRaWAN on IoT-LAB using RIOT

The goal of this notebook is to discover the basics of LoRaWAN communication using RIOT on IoT-LAB and TheThingsNetwork (TTN) as LoRaWAN network provider. This Notebook is divided in several steps:
1. The first step requires you to configure a LoRaWAN application with one device on the TTN backend
2. Then you will submit an experiment with one ST B-L072Z-LRWAN1 node (known as st-lrwan1) on the Saclay site
3. Once the experiment is running, you will build and flash a RIOT application that provides a shell to control the LoRaWAN stack. On the board, you will configure the identifiers and keys required for Over-The-Air Activation (OTAA).
4. In the last step of this tutorial, you will exchange messages between the board and TTN.

### LoRaWAN overview

LoRaWAN is a set of protocols built on top of the LoRa radio technology. It allows end devices to send messages on long distances and with potentially a very low power consumption.

The LoRa technology uses a Chirp Spreading Spectrum frequency modulation within public radio frequency bands (ISM). The access to the physical layer is regulated and depends on the region of use. On IoT-LAB, the EU868 ISM band is used by the boards, because they are all located in Europe and the boards can only transmit on this band.

<img src="images/lorawan-classes.jpg" width="640" align="center"/>

The LoRaWAN specifications define 3 classes of end-devices: A, B, C. Each of which has specific access to the physical layer. For more details, please refer to the LoRa specifications provided by the LoRa Alliance [here](https://www.lora-alliance.org/lorawan-for-developers).

Here is a global overview of a LoRaWAN network:

<img src="images/lorawan-system.png" width="640" align="center"/>

End devices use LoRa modulation to communicate with the gateways, which themselves use the regular Internet protocols to communicate with the LoRaWAN server, owned by a provider.

A TTN gateway is installed in the IoT-LAB Saclay site and connected to the TTN backend server. Thus, this allows LoRa devices deployed in the Saclay site to communicate with the TTN backend.

### Configure a TTN application

Before using the LoRa devices in IoT-LAB, you have to create an account and register an application with a device configured on TTN.

1. If you don’t already have a TTN account, create one from [here](https://account.thethingsnetwork.org/register)

<img src="images/ttn-signup.png" width="640" align="center"/>

After submitting the account creation form, check your emails and then activate the account.

2. Add an application to your TTN account by following [the official documentation](https://www.thethingsnetwork.org/docs/applications/add.html).

3. Register a device in your TTN application by following [this documentation](https://www.thethingsnetwork.org/docs/devices/registration.html).
   Keep the default Other-The-Air Activation (OTAA) procedure.The OTAA activation requires 3 informations:
   - **Device EUI**: the device unique identifier is a 8 bytes array (16 hex char string)
   - **Application EUI**: the application unique identifier is a 8 bytes array (16 hex char string)
   - **Application Key**: the application key is a 16 bytes array (32 hex char string)

**Important:** In the following instructions of this tutorial, we consider that your application name is **_iotlab-lorawan_** and your node id is **_iotlab-node_**. Replace these occurences with the information you configured in your TTN account.

### Book and configure one LoRa device

Now let's book one the LoRa device available in the IoT-LAB testbed. The boards are [ST B-L072Z-LRWAN1](https://www.st.com/en/evaluation-tools/b-l072z-lrwan1.html):

<img src="images/en.b-l072z-lrwan1.jpg" width="320" align="center"/>

1. Submit an experiment with one LoRa device on IoT-LAB:

In [ ]:
!iotlab-experiment submit -n riot_ttn -d 60 -l 1,archi=st-lrwan1:sx1276+site=saclay

In [ ]:
!iotlab-experiment wait

2. Build the `tests/pkg_semtech-loramac` application of RIOT.

This application provides a shell to finely control the loramac stack on the device. By default, the target of the application corresponds to the `st-lrwan1` board type in IoT-LAB.

In [ ]:
!make -C ../RIOT/tests/pkg_semtech-loramac

3. Flash the RIOT firmware on the board:

In [ ]:
!make IOTLAB_NODE=auto-ssh -C ../RIOT/tests/pkg_semtech-loramac flash-only

4. Open a terminal: `File > New > Terminal` and connect to the shell using the following command:

```sh
make IOTLAB_NODE=auto-ssh -C riot/RIOT/tests/pkg_semtech-loramac term
```

The shell provides the `loramac` command to control the loramac stack, join the network, and send packets:

```
> loramac
loramac
Usage: loramac <get|set|join|tx|link_check|save|erase>
```

5. In the RIOT shell, configure the loramac stack with the DEVEUI, APPEUI and APPKEY:

```
> loramac set deveui 00000000000000
> loramac set appeui 00000000000000
> loramac set appkey 0000000000000000000000000000
```

**note:** Use the values of your registered device in the TTN backend.

You can also set a fast datarate, e.g. 5, corresponding to a bandwidth of 125kHz and a spreading factor of 7, since the nodes are very close to the gateway:

```
> loramac set dr 5
```

6. Now that the device is correctly configured for OTAA activation, it is time to join it to the network:

```
> loramac join otaa
Join procedure succeeded!
```

On the TTN web console, go to the **Data** tab of the `iotlab-node` page. You should see the activation message received by the TTN backend (the item is clickable).

7. Now use the `tx` subcommand to send a payload to the backend: 

```
> loramac tx HelloWorld!
TX done
```

Still in the **Data** tab of the `iotlab-node` web page, you should see the message received by the TTN backend.

8. Send a downlink message from the **Overview** tab of the `iotlab-node` (for example use the following hexadecimal values: **48656C6C6F52494F5421**)

**Note**: Nothing is received by the node because a LoRaWAN class A device (the default for a RIOT LoRaWAN node) only have a short RX window after a send.

In order to receive the downlink message, just send another message to TTN: 

```
loramac tx HelloWorld!
Data received: HelloRIOT!, port: 1
```

You sent and received LoRaWAN messages using RIOT on IoT-LAB, congratulations !